<a href="https://colab.research.google.com/github/Chu-Yichen/BASC0017-CW1/blob/main/Figure_2_Emission_and_Utility_Profile_Computation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize


# 1. PARAMETERS
alpha_UK    = 100.0
alpha_USA   = 200.0
alpha_China = 400.0
alpha_India = 200.0

d_UK    = 1.42
d_USA   = 1.48
d_China = 1.66
d_India = 2.12

e0_UK    = 0.31
e0_USA   = 5.08
e0_China = 11.35
e0_India = 2.83

alpha_params = (alpha_UK, alpha_USA, alpha_China, alpha_India)
d_params     = (d_UK, d_USA, d_China, d_India)
e0_params    = (e0_UK, e0_USA, e0_China, e0_India)
countries    = ["UK", "USA", "China", "India"]

# 2. UTILITY FUNCTIONS
def Ui(e, i):
    """
    Compute the utility for country i given the emissions profile e = (e_UK, e_USA, e_China, e_India).
    Payoff function:
      U_i(e_i, E) = α_i * sqrt(e_i) - (d_i/4) * (E)^(3/2),
    where E = sum(e).
    """
    alpha = [alpha_UK, alpha_USA, alpha_China, alpha_India]
    d     = [d_UK, d_USA, d_China, d_India]
    E = sum(e)
    return alpha[i] * math.sqrt(e[i]) - (d[i] * (E ** 1.5)) / 4

def total_utility(e): # Compute aggregate utility across all countries.
    return sum(Ui(e, i) for i in range(4))

# 3. NASH EQUILIBRIUM (Iterative Best Response)
def best_response(e_others, alpha_i, d_i, e0_i):
    """
    For country i, given the sum of other emissions (e_others),
    solve for the optimal emission e_i.
    Substitute x = sqrt(e_i) so that the first-order condition becomes:
      F(x) = α_i/(2x) - (3*d_i)/(8)*√(x² + e_others) = 0.
    Solve for x* via the bisection method and return e_i = (x*)²,
    clamping to the baseline e0_i if the solution is lower.
    """
    def F(x):
        return alpha_i/(2*x) - (3*d_i)/(8) * math.sqrt(x*x + e_others)

    L, R = 1e-8, 1.0
    while F(R) > 0 and R < 1e6:
        R *= 2.0
    if F(L)*F(R) > 0:
        return e0_i  # No sign change; return baseline

    for _ in range(100):
        mid = 0.5*(L+R)
        if F(mid) == 0:
            L = R = mid
            break
        elif F(mid) > 0:
            L = mid
        else:
            R = mid
    x_star = 0.5*(L+R)
    e_star = x_star*x_star
    return max(e_star, e0_i)

def find_nash_equilibrium(alpha_p, d_p, e0_p, tol=1e-7, max_iter=10000):
    """
    Iteratively update each country's emission using its best response until convergence.
    Returns the Nash equilibrium emission profile as a tuple.
    """
    e = list(e0_p)  # Initialize with baseline emissions.
    for _ in range(max_iter):
        old = e[:]
        sums = [sum(e) - e[i] for i in range(4)]  # Compute E_{-i} for each country.
        e[0] = best_response(sums[0], alpha_p[0], d_p[0], e0_p[0])
        e[1] = best_response(sums[1], alpha_p[1], d_p[1], e0_p[1])
        e[2] = best_response(sums[2], alpha_p[2], d_p[2], e0_p[2])
        e[3] = best_response(sums[3], alpha_p[3], d_p[3], e0_p[3])
        if max(abs(e[i]-old[i]) for i in range(4)) < tol:
            break
    return tuple(e)


# 4. SOCIAL OPTIMUM (Global Optimization)
def neg_total_utility(e):
    """Objective function for minimization: negative of total utility."""
    return -total_utility(e)

constraints = [{'type': 'ineq', 'fun': lambda x, i=i: x[i] - e0_params[i]} for i in range(4)]
x0 = [2.0, 2.0, 3.0, 3.0]  # Initial guess (above baselines)
res = minimize(neg_total_utility, x0=x0, constraints=constraints, method='SLSQP', options={'ftol':1e-9, 'maxiter':10000})
social_opt = res.x
tu_social = total_utility(social_opt)


# 5. NASH EQUILIBRIUM
nash_eq = find_nash_equilibrium(alpha_params, d_params, e0_params)
tu_nash = total_utility(nash_eq)


# 6. THE RESULTS TABLE
# Compute each country's data.
nash_emissions = list(nash_eq)
social_emissions = list(social_opt)
nash_utilities = [Ui(nash_eq, i) for i in range(4)]
social_utilities = [Ui(social_opt, i) for i in range(4)]
utility_diff = [social_utilities[i] - nash_utilities[i] for i in range(4)]

# Create a DataFrame for individual countries.
df = pd.DataFrame({
    "Country": countries,
    "Nash Emissions (GtCO₂)": nash_emissions,
    "Social Emissions (GtCO₂)": social_emissions,
    "Nash Utility": nash_utilities,
    "Social Utility": social_utilities,
    "Utility Diff (Social-Nash)": utility_diff
})

# Create a "Total" row that sums numeric columns.
total_row = {
    "Country": "Total",
    "Nash Emissions (GtCO₂)": sum(nash_emissions),
    "Social Emissions (GtCO₂)": sum(social_emissions),
    "Nash Utility": sum(nash_utilities),
    "Social Utility": sum(social_utilities),
    "Utility Diff (Social-Nash)": sum(utility_diff)
}
df = pd.concat([df, pd.DataFrame([total_row])], ignore_index=True)

# 7. STYLE THE DATAFRAME
def row_color(row):
    """
    Return a list of CSS styles for the entire row based on the Country.
    Assign a distinct color for each country (and a default for "Total").
    """
    color_map = {
        "UK": "background-color: lightblue",
        "USA": "background-color: lightcoral",
        "China": "background-color: lightgoldenrodyellow",
        "India": "background-color: lightgreen",
        "Total": "background-color: lightgray"
    }
    return [color_map.get(row["Country"], "")] * len(row)

styled_df = (df.style
             .apply(row_color, axis=1)  # Apply row-level coloring
             .format("{:.2f}", subset=["Nash Emissions (GtCO₂)", "Social Emissions (GtCO₂)", "Nash Utility", "Social Utility", "Utility Diff (Social-Nash)"])
             .hide(axis="index")
             .set_properties(**{
                 'text-align': 'center',
                 'font-family': 'Arial',
                 'border': '1px solid lightgrey'
             })
             .set_caption("Figure 2: Emission and Utility Profiles by Country (Nash Equilibrium Vs. Soical Optimal)")
             .set_table_styles([{
                 'selector': 'caption',
                 'props': [('font-size', '14pt'),
                           ('font-weight', 'bold'),
                           ('color', '#2e6e8e')]
             }])
            )

display(styled_df)


Country,Nash Emissions (GtCO₂),Social Emissions (GtCO₂),Nash Utility,Social Utility,Utility Diff (Social-Nash)
UK,22.02,3.99,-2375.12,-154.14,2220.98
USA,81.08,15.97,-1163.67,430.31,1593.98
China,257.80,63.87,3097.32,2783.05,-314.27
India,39.51,15.97,-2989.29,270.79,3260.08
Total,400.41,99.80,-3430.76,3330.00,6760.77
